In [ ]:
"""
@ Purpose:
    YOLOv5Seg에서 저장되는 save_txt를 실제로 그림에 표시해보기
    save_txt 시에 Polygon 형태로 저장되며, masks2segments 함수에서 알 수 있듯이 기본적으로 largest만을 저장한다 (Concat도 선택 가능)
    데이터가 매우 크며, 추후 RLE 형태로 저장할 필요가 있음
"""

In [11]:
# import libraries
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import cv2
# interact
from ipywidgets import interact

In [12]:
VIDEO_PATH = '../sources/KIMPO_230808_4.MOV'
LABEL_PATH = '../runs/predict-seg/exp/labels'

In [13]:
# Exists check
if not os.path.exists(VIDEO_PATH):
    print('Video file not exists')
    sys.exit()
if not os.path.exists(LABEL_PATH):
    print('Label file not exists')
    sys.exit()

In [14]:
cap = cv2.VideoCapture(VIDEO_PATH)
max_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
cap.release()

In [21]:
@interact(frame_id=(1, max_frame))
def plot_frame(frame_id):
    cap = cv2.VideoCapture(VIDEO_PATH)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id-1)
    ret, frame = cap.read()
    if not ret:
        print("프레임을 가져오는데 실패했습니다.")
        cap.release()
        return

    file_name = os.path.splitext(os.path.basename(VIDEO_PATH))[0]
    label_file = os.path.join(LABEL_PATH, file_name + '_' + str(frame_id) + '.txt')

    # 레이블 파일 읽기 및 포인트 조정
    if os.path.exists(label_file):
        with open(label_file, 'r') as f:
            data = f.read().strip()

        for line in data.split('\n'):
            parts = line.split()
            label = int(parts[0])
            points = np.array(parts[1:], dtype=float).reshape(-1, 2)
            points[:, 0] *= frame.shape[1]  # X 좌표 조정
            points[:, 1] *= frame.shape[0]  # Y 좌표 조정
            # put random color
            color = np.random.randint(0, 255, 3).tolist()
            
            cv2.polylines(frame, [points.astype(np.int32)], True, color, 7)

    # 프레임 표시
    plt.figure(figsize=(14, 10))
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    cap.release()

interactive(children=(IntSlider(value=612, description='frame_id', max=1224, min=1), Output()), _dom_classes=(…